## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

C:\Users\kajal\miniconda3\envs\pytorch\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


## Set Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Create Fully Connected Network

In [3]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(20*7*7, num_classes)
        
    # Forward method
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        return x

In [4]:
def save_checkpoint(state, filename='my_checkpoint.pth.tar'):
    print("=> Saving checkpoints")
    torch.save(state, filename)
    
def load_checkpoint(checkpoint):
    print("=> Loading Checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [5]:
## Check on random data
# model = CNN(3, 10)
# x = torch.randn(74,3,32,32)
# print(model(x).shape)

## Hyperparameters

In [6]:
in_channels = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 10
load_model = True    # lets add another hyperparameter

## Load Data

In [7]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform= transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='/dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## Initialize Network

In [8]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

## Loss and Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Now need to load the checkpoint file using the load_model hyperparameter
if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.tar'))

=> Loading Checkpoint


## Train Network

In [11]:
## To save and load model, we need to add check_points as dictionary
## need to create this checkpoint function above

for epoch in range(num_epochs):
    losses = []
    
    #lets say we are saving the epochs after every 'n'(2) epoch
    if epoch % 2 == 0:                               # at epoch 2 will write as, epoch == 2":
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}     # we can save other parameters also, like lr, epochs etc
        save_checkpoint(checkpoint)
    
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
#         print(data.shape)           #([64, 1, 28, 28])
        #but we want single 784 vector
#         data = data.reshape(data.shape[0], -1)
#         print(data.shape)
        
        # Forward
        pred = model(data)
        loss = criterion(pred, targets)
        losses.append(loss.item())
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient descent step
        optimizer.step()
        
    mean_loss = sum(losses)/len(losses)
    print(f'Loss at epoch {epoch} was {mean_loss:.5f}')

=> Saving checkpoints
Loss at epoch 0 was 5.78314
Loss at epoch 1 was 5.62543
=> Saving checkpoints
Loss at epoch 2 was 5.47326
Loss at epoch 3 was 5.32902
=> Saving checkpoints
Loss at epoch 4 was 5.19380
Loss at epoch 5 was 5.06688
=> Saving checkpoints
Loss at epoch 6 was 4.94813
Loss at epoch 7 was 4.83475
=> Saving checkpoints
Loss at epoch 8 was 4.72805
Loss at epoch 9 was 4.62700


## Check Accuracy

In [12]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on Training data")
    else:
        print("Checking accuracy on Test data")

    num_correct = 0
    num_samples = 0
    model.eval()           # set model in evaluation mode
    
    with torch.no_grad():   #When we are checking accuracy, no need to find gradient
        for x , y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
#             x = x.reshape(x.shape[0], -1)
            
            pred = model(x)
            _, predictions = pred.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)     #of 1st dimension
            
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()
    

In [13]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training data
Got 1949/60000 with accuracy 3.25
Checking accuracy on Test data
Got 376/10000 with accuracy 3.76
